In [1]:
Float = Float64; # Define datatype to be used for all operations

In [2]:
using Printf
using ProgressMeter
using SymPy
using Base.Iterators
using MLUtils
ProgressMeter.ijulia_behavior(:clear)

false

In [3]:
using DelimitedFiles
using Random
using Statistics
using Plots
using Flux
using LinearAlgebra

# Create the Neural Network

#### Define the network architecture

In [4]:
slow_sigmoid(x::Number) = 1 ./ (1 .+ exp.(.-x))

slow_sigmoid (generic function with 1 method)

In [5]:
model = Chain(
  Dense(20 * 20, 300, slow_sigmoid),       
  Dense(300, 4, slow_sigmoid),                        
)
model

Chain(
  Dense(400 => 300, slow_sigmoid),      # 120_300 parameters
  Dense(300 => 4, slow_sigmoid),        # 1_204 parameters
)                   # Total: 4 arrays, 121_504 parameters, 474.875 KiB.

In [6]:
#set all the parameters type to be float64 instead of float32
model = fmap(f64, model)

Chain(
  Dense(400 => 300, slow_sigmoid),      # 120_300 parameters
  Dense(300 => 4, slow_sigmoid),        # 1_204 parameters
)                   # Total: 4 arrays, 121_504 parameters, 949.500 KiB.

## Set the weights that were already created in python

In [10]:
using NPZ
#load the weights
W1 = NPZ.npzread("model_weights/weights_layer1.npy")
W2 = NPZ.npzread("model_weights/weights_layer2.npy")

4×300 Matrix{Float64}:
  0.213306  -0.315031  -0.440292   …   0.242114    0.0852821   0.746295
  0.219361  -0.902037  -0.296477       0.20508    -0.0750829  -0.0438323
 -1.22548    0.898655   0.0975446      0.0454408  -1.01628    -0.587954
  0.201931  -0.671274  -0.55822       -0.94544    -0.262693   -0.293441

In [11]:
#set weights
model.layers[1].weight .= W1
model.layers[2].weight .= W2

#set biases to 0
model.layers[1].bias .= zeros(300)  # Zero bias for first layer
model.layers[2].bias .= zeros(4)    # Zero bias for second layer

4-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0

# Test the Model

## some examples for comparison

In [12]:
function transform_input(input, new_size)
    new_image = fill(-1.0f0, new_size...)  # Create new array filled with -1.0
    input = reshape(input, 5, 5)  # Reshape input to 5x5
    rows, cols = size(input)  # Get size of the reshaped input

    # Calculate the starting indices to center the 5x5 image in the new array
    start_row = div(new_size[1] - rows, 2)
    start_col = div(new_size[2] - cols, 2)

    # Place the 5x5 image in the center of the new image
    new_image[start_row+1:start_row+rows, start_col+1:start_col+cols] .= input

    return new_image
end


transform_input (generic function with 1 method)

In [13]:
forecast = [0, 0, 0, -0.2759131522179011, -0.37265483654235754, 0, -0.12257477450946719, 0.22259646612114253, 0, -0.2559704329900314, 0, 0, 0, 0, 0, 0, 0, 0.02067362885974433, 0, 0.02406361296189477, 0, 0, -0.00805149918363896, 0, 0]

input = transform_input(forecast, (20,20))
print(input)

Float32[-1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0; -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0; -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0; -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0; -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0; -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0; -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0; -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 0.0 0.0 0.0 0.0 0.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0; -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 0.0 -0.12257478 0.0 0.0 0.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0; -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 -1.0 0.0 0.22259647 0.0 0.020673629 -0.008051499 -1.0 -

In [14]:
output = model(vec(input))
print(output)

[3.4545493082816434e-14, 2.405282442474008e-24, 4.1871809143590725e-5, 1.76377217731836e-11]

# Define Record Type for Formula Extraction

We now define a data type that we will send through the network instead of a floating point number. The new data type wraps another data type (e.g. a float) but also records everything that happens to it and generates a formula for the value that it wraps while it is manipulated.

We can then send a matrix of instances of this data type through the network and the output values will not only be the correct neuron outputs, but also a formula of how we got to this result.

More info can be found in the white paper.

In [16]:
"""
Recursively joins strings and Arrays of String
"""
function joinrecurse(a)
    if typeof(a) <: AbstractString
        return a
    end
    return join([joinrecurse(i) for i in a])
end

joinrecurse

In [17]:
struct RecordFormula{T<:Real} <: Number
    val::T
    formula::Vector{Any}
end

RecordFormula(v::T, f::S) where{T<:Real, S} = RecordFormula{T}(v, f)
RecordFormula{S}(v::T) where {S, T<:Real} = RecordFormula{S}(S(v), [string(v)])
RecordFormula(v::T) where {T<:Real} = RecordFormula{T}(v, [string(v)])
# Constructor that converts Int to Float64
RecordFormula(v::Int) = RecordFormula{Float64}(Float64(v), [string(v)])
RecordFormula(v::Bool) = RecordFormula{Int}(Int(v))
val(r::RecordFormula) = r.val
formula(r::RecordFormula) = joinrecurse(r.formula)
Base.Pair(r::RecordFormula) = Pair(formula(r)=>val(r))

Base.similar(a::Array{T}, m::Int) where {T<:RecordFormula} = zeros(T, m)
Base.similar(a::AbstractArray{T}, dims::Base.DimOrInd...) where {T<:RecordFormula} = zeros(T, Base.to_shape(dims))
Base.similar(a::Array{T}, dims::Dims{N}) where {T<:RecordFormula,N} = zeros(T, dims)
Base.similar(a::AbstractArray, ::Type{T}, dims::Base.DimOrInd...) where {T<:RecordFormula} = zeros(T, Base.to_shape(dims))

Base.typemin(t::Type{RecordFormula{T}}) where {T} = typemin(T)
Base.typemin(x::RecordFormula{T}) where {T} = typemin(T)

#define cunjunction to avoid error
conj(r::RecordFormula{T}) where {T<:Real} = r

Base.convert(::Type{RecordFormula{T}}, v::T) where {T} = RecordFormula{T}(v, [string(v)])
Base.convert(::Type{RecordFormula{S}}, v::RecordFormula{T}) where {S, T} = RecordFormula{S}(S(val(v)), v.formula)
Base.convert(::Type{RecordFormula{T}}, x::Number) where {T} = RecordFormula(x)
Base.promote_rule(::Type, ::Type{RecordFormula{T}}) where {T} = RecordFormula{T}
Base.promote_rule(::Type{RecordFormula{S}}, ::Type{RecordFormula{T}}) where {S,T} = RecordFormula{promote_type(S,T)}
Base.promote_rule(::Type{Bool}, ::Type{RecordFormula{T}}) where {T} = RecordFormula{T}

Base.:(==)(a::RecordFormula, b::RecordFormula) = val(a) == val(b)

recordtostring(r::RecordFormula) = formula(r)*";"*@sprintf("%.20g", val(r))

recordtostring (generic function with 1 method)

In [18]:
for op in [:*, :+, :-, :/]
    @eval begin
        function Base.$op(a::RecordFormula, b::RecordFormula)
            str = ["(", a.formula, string($op), b.formula, ")"]
            newval = $op(a.val, b.val)
            
            return RecordFormula(newval, str)
        end
    end
end

for op in [:sin, :cos, :tan, :tanh, :log, :exp, :-, :abs]
    @eval begin
        function Base.$op(a::RecordFormula)
            str = [string($op), "(", a.formula, ")"]
            newval = $op(a.val)
            
            return RecordFormula(newval, str)
        end
    end
end

In [19]:
function Base.max(a::RecordFormula, b::RecordFormula)
    v = max(val(a), val(b))
    return RecordFormula(v, ["(",formula(a)," max ", formula(b),")"])
end

Base.max(a::Real, b::RecordFormula) = max(RecordFormula(a), b)

### Define functions to wrap elements of tensors/arrays into RecordFormulas to be passed through a neural network

In [20]:
function recordify(a::Array{T, 1}, name::String) where T
    res = Array{RecordFormula, 1}(size(a))
    for r in 1:size(a, 1)
        res[r] = RecordFormula(a[r], ["$(name)_$(r)_"])
    end
    return res
end

recordify (generic function with 1 method)

In [21]:
function recordify(a::Array{T, 2}, name::String) where T
    res = Array{RecordFormula, 2}(size(a))
    for c in 1:size(a, 2)
        for r in 1:size(a, 1)
            res[r,c] = RecordFormula(a[r,c], ["$(name)_$(r)_$(c)_"])
        end
    end
    return res
end

recordify (generic function with 2 methods)

In [22]:
function recordify(a::Array{T, 3}, name::String) where T
    res = Array{RecordFormula, 3}(size(a))
    for k in 1:size(a, 3), j in 1:size(a, 2), i in 1:size(a, 1)
        res[i,j,k] = RecordFormula(a[i,j,k], ["$(name)_$(i)_$(j)_$(k)_"])
    end
    return res
end

recordify (generic function with 3 methods)

In [23]:
function recordify(a::Array{T, 4}, name::String) where T
    res = Array{RecordFormula, 4}(size(a))
    for l in 1:size(a, 4), k in 1:size(a, 3), j in 1:size(a, 2), i in 1:size(a, 1)
        res[i,j,k,l] = RecordFormula(a[i,j,k,l], ["$(name)_$(i)_$(j)_$(k)_$(l)_"])
    end
    return res
end

recordify (generic function with 4 methods)

# Extract Formulas
Define functions and helper functions to extract layerwise formulas from a neural networks

In [24]:
"""Given the output of a layer, returns formulas for all the outputs and a new array where the formulas are replaced by intermediate variabels"""
function generate_layer_formula(layer_out, layer_prefix)
    res = copy(layer_out)
    v = @view res[:]
    formulas = []
    for i in 1:length(v)
        var_name = "$(layer_prefix)_$i"
        f = formula(v[i])
        push!(formulas, "$var_name=$f")
        v[i] = RecordFormula(val(v[i]), ["$(layer_prefix)_$i"])
    end
        
    return formulas, res
end

generate_layer_formula

In [25]:
"""Given an input image, creates an input with input variable placeholders to create formulas"""
function generate_input(img)
    o = map(x->RecordFormula(x[2], ["in_$(x[1])"]), enumerate(img))
    return reshape(o, (400, 1))
end

generate_input

In [26]:
"""Given a model and an input, returns all formulas for all the layers of it"""
function formalize_model(model, input)
    input = Flux.flatten(input)
    #input = input'  # Transpose the input so that the shape is correct
    all_formulas = []
    max_layers = length(model)
    @showprogress for (i, layer) in enumerate(model)
        model_out = layer(input)
        layer_output_name = i == max_layers ? "out" : "l$i"
        if i == 1
            print(model_out)
        end
        formulas, input = generate_layer_formula(model_out, layer_output_name)
        push!(all_formulas, formulas)
    end
    return all_formulas
end

formalize_model

In [27]:
"""Given an image, converts it to input assignments for the formulas and expected outputs, used for testing"""
function convert_image(img, model)
    img = Flux.flatten(img)
    input = ["in_$i=$val" for (i, val) in enumerate(vec(img[:,:,:,:]))]
    model_output = model(reshape(img, (400, 1)))
    output = ["out_$i=$val" for (i, val) in enumerate(vec(model_output))]
    return input, output
end

convert_image

In [35]:
training_sample = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,0,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,18,57,66,13,0,0,1,0,0,0,0,0,0,0,0,0,1,0,31,164,231,255,255,229,136,5,1,1,0,0,0,0,0,0,0,2,1,33,224,255,255,254,253,255,255,186,6,2,1,0,0,0,0,0,0,3,0,152,255,248,254,255,255,254,249,255,126,0,3,0,0,0,0,0,2,0,31,248,254,254,255,255,255,255,253,253,237,17,0,1,0,0,0,0,3,0,75,255,252,255,255,255,255,255,255,254,255,58,0,3,0,0,0,0,3,0,75,255,252,255,255,255,255,255,255,254,255,62,0,3,0,0,0,0,1,0,17,232,253,253,255,255,255,255,254,253,244,24,0,2,0,0,0,0,0,4,0,125,255,248,254,255,255,254,248,255,159,0,3,0,0,0,0,0,0,1,2,13,185,255,255,255,253,255,255,210,28,2,2,0,0,0,0,0,0,0,1,0,7,132,233,254,255,242,146,17,0,1,0,0,0,0,0,0,0,0,0,1,0,0,17,44,64,26,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,3,0,0,0,0,3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

400-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [36]:
#testing
input = generate_input(training_sample)
input = Flux.flatten(input)
#input_t = input'  # Transpose the input so that the shape is correct
print(input)

RecordFormula{Int64}[RecordFormula{Int64}(0, Any["in_1"]); RecordFormula{Int64}(0, Any["in_2"]); RecordFormula{Int64}(0, Any["in_3"]); RecordFormula{Int64}(0, Any["in_4"]); RecordFormula{Int64}(0, Any["in_5"]); RecordFormula{Int64}(0, Any["in_6"]); RecordFormula{Int64}(0, Any["in_7"]); RecordFormula{Int64}(0, Any["in_8"]); RecordFormula{Int64}(0, Any["in_9"]); RecordFormula{Int64}(0, Any["in_10"]); RecordFormula{Int64}(0, Any["in_11"]); RecordFormula{Int64}(0, Any["in_12"]); RecordFormula{Int64}(0, Any["in_13"]); RecordFormula{Int64}(0, Any["in_14"]); RecordFormula{Int64}(0, Any["in_15"]); RecordFormula{Int64}(0, Any["in_16"]); RecordFormula{Int64}(0, Any["in_17"]); RecordFormula{Int64}(0, Any["in_18"]); RecordFormula{Int64}(0, Any["in_19"]); RecordFormula{Int64}(0, Any["in_20"]); RecordFormula{Int64}(0, Any["in_21"]); RecordFormula{Int64}(0, Any["in_22"]); RecordFormula{Int64}(0, Any["in_23"]); RecordFormula{Int64}(0, Any["in_24"]); RecordFormula{Int64}(0, Any["in_25"]); RecordFormula

In [37]:
# Generate formulas for the newtork using an arbitrary image as a dummy for input conversion
all_formulas = formalize_model(model, generate_input(training_sample));

RecordFormula{Float64}[RecordFormula{Float64}(0.0, Any["(", Any["1"], "/", Any["(", Any["1"], "+", Any["exp", "(", Any["-", "(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["(", Any["("

Excessive output truncated after 524289 bytes.

"-0.017650201285148135"], "*", Any["in_341"], ")"], ")"], "+", Any["(", Any["-0.02858577324166848"], "*", Any["in_342"], ")"], ")"], "+", Any["(", Any["-0.046040674320204734"], "*", Any["in_343"], ")"], ")"], "+", Any["(", Any["0.05072469361879353"], "*", Any["in_344"], ")"], ")"], "+", Any["(", Any["-0.029623307630867318"], "*", Any["in_345"], ")"], ")"], "+", Any["(", Any["0.06255948104051223"], "*", Any["in_346"], ")"], ")"], "+", Any["(", Any["0.048554922408826244"], "*", Any["in_347"], ")"], ")"], "+", Any["(", Any["-0.013106729582730564"], "*", Any["in_348"], ")"], ")"], "+", Any["(", Any["-0.010339861260526826"], "*", Any["in_349"], ")"], ")"], "+", Any["(", Any["-0.07985916999951354"], "*", Any["in_350"], ")"], ")"], "+", Any["(", Any["0.01394841631102575"], "*", Any["in_351"], ")"], ")"], "+", Any["(", Any["0.005363213725416035"], "*", Any["in_352"], ")"], ")"], "+", Any["(", Any["0.020035535165937396"], "*", Any["in_353"], ")"], ")"], "+", Any["(", Any["0.009616457402953478"]

In [44]:
i, o = convert_image(training_sample, model)

(["in_1=0", "in_2=0", "in_3=0", "in_4=0", "in_5=0", "in_6=0", "in_7=0", "in_8=0", "in_9=0", "in_10=0"  …  "in_391=0", "in_392=0", "in_393=0", "in_394=0", "in_395=0", "in_396=0", "in_397=0", "in_398=0", "in_399=0", "in_400=0"], ["out_1=0.9990000712587231", "out_2=0.0026791365374065667", "out_3=0.005965406883159599", "out_4=0.008432244993845213"])

### Generate output files for the network as required by Paceval
Docstrings are part of an email from Jörg, defining the required output format

In [38]:
"""
Deine Funktionen müssten dann jede in einer separaten Datei sein (anstatt in einer grossen).
In der Datei dann nur die Funktion, d.h. nicht " l1_1=". Der Titel der Textdatei der 
Funktion müsste dann mit "l1_1.txt", "l1_10.txt", " l1_11.txt" usw. benannt werden bis dann 
"out_1.txt", " out_2.txt" usw.
"""
function write_formulas(all_formulas, base_dir)
    formulas = Base.Iterators.flatten(all_formulas)|>collect
    for (varname, formula) in split.(formulas, "=")
        open(joinpath(base_dir, varname*".txt"), "w") do io
            println(io, formula)
        end
    end
end

write_formulas

In [39]:
"""
Alle Variablen, die in den Funktionen verwendet werden (Deine "in_1 .. in_784"),
müssten in eine Datei, s. "paceval_CNN_variablesString.txt" durch Leerzeichen getrennt.
"""
get_input_variable_names(example_input_formulas) = first.(split.(example_input_formulas, "="))

function write_input_variable_names(example_input_formulas, filename)
    open(filename, "w") do io
        variable_names = get_input_variable_names(example_input_formulas)
        println(io, join(variable_names, " "))
    end
end

write_input_variable_names (generic function with 1 method)

In [40]:
"""
Alle Layer, die es gibt (Deine "l1_1 .. l2_16") müssten ebenfalls in eine Datei
durch Leerzeichen getrennt, und zwar in der Reihenfolge, wie Sie abgearbeitet werden.
Wenn Layer parallelisiert, werden können dann fasse diese in einer Zeile zusammen
"""
function get_layer_variable_names(all_formulas)
    result_layers = []
    for layer in all_formulas
        layer_variable_names = join(first.(split.(layer, "=")), " ")
        push!(result_layers, layer_variable_names)
    end
    result_layers
end

function write_layer_variable_names(all_formulas, filename)
    open(filename, "w") do io
        layer_variable_names = get_layer_variable_names(all_formulas)
        for layer in layer_variable_names
            println(io, layer)
        end
    end
end

write_layer_variable_names (generic function with 1 method)

### Generate formulas for paceval

In [45]:
write_formulas(all_formulas, "./formulas")
write_input_variable_names(i, "formulas/paceval_variablesString.txt")
write_layer_variable_names(all_formulas, "formulas/paceval_layerVariableNames.txt")